In [556]:
import time
import numpy as np
import psi4
import sys

In [2]:
class helper_diis(object):
    def __init__(self, t1, t2, max_diis):

        self.oldt1 = t1.copy()
        self.oldt2 = t2.copy()
        self.diis_vals_t1 = [t1.copy()]
        self.diis_vals_t2 = [t2.copy()]
        self.diis_errors = []
        self.diis_size = 0
        self.max_diis = max_diis

    def add_error_vector(self, t1, t2):

        # Add DIIS vectors
        self.diis_vals_t1.append(t1.copy())
        self.diis_vals_t2.append(t2.copy())
        # Add new error vectors
        error_t1 = (self.diis_vals_t1[-1] - self.oldt1).ravel()
        error_t2 = (self.diis_vals_t2[-1] - self.oldt2).ravel()
        self.diis_errors.append(np.concatenate((error_t1, error_t2)))
        self.oldt1 = t1.copy()
        self.oldt2 = t2.copy()

    def extrapolate(self, t1, t2):

        # Limit size of DIIS vector
        if (len(self.diis_vals_t1) > self.max_diis):
            del self.diis_vals_t1[0]
            del self.diis_vals_t2[0]
            del self.diis_errors[0]

        self.diis_size = len(self.diis_vals_t1) - 1

        # Build error matrix B
        B = np.ones((self.diis_size + 1, self.diis_size + 1)) * -1
        B[-1, -1] = 0

        for n1, e1 in enumerate(self.diis_errors):
            B[n1, n1] = np.dot(e1, e1)
            for n2, e2 in enumerate(self.diis_errors):
                if n1 >= n2: continue
                B[n1, n2] = np.dot(e1, e2)
                B[n2, n1] = B[n1, n2]

        B[:-1, :-1] /= np.abs(B[:-1, :-1]).max()

        # Build residual vector
        resid = np.zeros(self.diis_size + 1)
        resid[-1] = -1

        # Solve pulay equations
        ci = np.linalg.solve(B, resid)

        # Calculate new amplitudes
        t1 = np.zeros_like(self.oldt1)
        t2 = np.zeros_like(self.oldt2)
        for num in range(self.diis_size):
            t1 += ci[num] * self.diis_vals_t1[num + 1]
            t2 += ci[num] * self.diis_vals_t2[num + 1]

        # Save extrapolated amplitudes to old_t amplitudes
        self.oldt1 = t1.copy()
        self.oldt2 = t2.copy()

        return t1, t2


In [671]:
class HelperCCEnergy(object):
    def __init__(self, mol, rhf_e, rhf_wfn, memory=2):

        print("\nInitalizing CCSD object...\n")

        # Integral generation from Psi4's MintsHelper
        time_init = time.time()

        self.rhf_e = rhf_e
        self.wfn = rhf_wfn

        self.ccsd_corr_e = 0.0
        self.ccsd_e = 0.0

        self.ndocc = self.wfn.doccpi()[0]
        self.nmo = self.wfn.nmo()
        self.memory = memory
        self.C = self.wfn.Ca()
        self.npC = np.asarray(self.C)

        self.mints = psi4.core.MintsHelper(self.wfn.basisset())
        H = np.asarray(self.mints.ao_kinetic()) + np.asarray(
            self.mints.ao_potential())
        self.nmo = H.shape[0]

        # Update H, transform to MO basis
        H = np.einsum('uj,vi,uv', self.npC, self.npC, H)

        print('Starting AO ->  MO transformation...')

        ERI_Size = self.nmo * 128.e-9
        memory_footprint = ERI_Size * 5
        if memory_footprint > self.memory:
            psi.clean()
            raise Exception(
                "Estimated memory utilization (%4.2f GB) exceeds numpy_memory \
                            limit of %4.2f GB." % (memory_footprint,
                                                   self.memory))

        # Integral generation from Psi4's MintsHelper
        self.MO = np.asarray(self.mints.mo_eri(self.C, self.C, self.C, self.C))
        # Physicist notation
        self.MO = self.MO.swapaxes(1,2)
        
        print("Size of the ERI tensor is %4.2f GB, %d basis functions." %
              (ERI_Size, self.nmo))

        # Update nocc and nvirt
        self.nocc = self.ndocc
        self.nvirt = self.nmo - self.nocc

        # Make slices
        self.slice_o = slice(0, self.nocc)
        self.slice_v = slice(self.nocc, self.nmo)
        self.slice_a = slice(0, self.nmo)
        self.slice_dict = {
            'o': self.slice_o,
            'v': self.slice_v,
            'a': self.slice_a
        }

        # Compute Fock matrix
        self.F = H + 2.0 * np.einsum('pmqm->pq',
                                     self.MO[:, self.slice_o, :, self.slice_o])
        self.F -= np.einsum('pmmq->pq',
                            self.MO[:, self.slice_o, self.slice_o, :])

        self.F = np.diag(np.diag(self.F))
        
        ### Occupied and Virtual orbital energies
        Focc = np.diag(self.F)[self.slice_o]
        Fvir = np.diag(self.F)[self.slice_v]

        self.Dia = Focc.reshape(-1, 1) - Fvir
        self.Dijab = Focc.reshape(-1, 1, 1, 1) + Focc.reshape(
            -1, 1, 1) - Fvir.reshape(-1, 1) - Fvir

        ### Construct initial guess
        print('Building initial guess...')
        # t^a_i
        self.t1 = np.zeros((self.nocc, self.nvirt))
        # t^{ab}_{ij}
        self.t2 = self.MO[self.slice_o, self.slice_o, self.slice_v,
                          self.slice_v] / self.Dijab

        print('\n..initialized CCSD in %.3f seconds.\n' %
              (time.time() - time_init))

    # occ orbitals  : i, j, k, l, m, n
    # virt orbitals : a, b, c, d, e, f
    # all oribitals : p, q, r, s, t, u, v

    def get_F(self, string):
        if len(string) != 2:
            psi4.core.clean()
            raise Exception('get_F: string %s must have 2 elements.' % string)
        return self.F[self.slice_dict[string[0]], self.slice_dict[string[1]]]
    
    def get_MO(self, string):
        if len(string) != 4:
            psi4.core.clean()
            raise Exception('get_MO: string %s must have 4 elements.' % string)
        return self.MO[self.slice_dict[string[0]], self.slice_dict[string[1]],
                       self.slice_dict[string[2]], self.slice_dict[string[3]]]

    def build_restricted_intermediates(self):
        #asym_t2 = 2*self.t2 - self.t2.swapaxes(0,1)
        asym_t2 = 2*self.t2 - self.t2.swapaxes(2,3)
    
        c_ijab = np.einsum('ia,jb->ijab',self.t1,self.t1)
        c_ijab += self.t2
    
        I_ai = 2*np.einsum('aeim,me->ai',self.get_MO('vvoo'),self.t1)
        I_ai -= np.einsum('eaim,me->ai',self.get_MO('vvoo'),self.t1)
        I_ai += self.get_F('vo')
    
        I_ba = 2*np.einsum('beam,me->ba',self.get_MO('vvvo'),self.t1)
        I_ba -= np.einsum('bema,me->ba',self.get_MO('vvov'),self.t1)
        I_ba -= 2*np.einsum('ebmn,mnea->ba',self.get_MO('vvoo'),c_ijab)
        I_ba += np.einsum('bemn,mnea->ba',self.get_MO('vvoo'),c_ijab)
        I_ba += self.get_F('vv')-np.diag(np.diag(self.get_F('vv')))
        I_ba -= np.einsum('ma,bm->ba',self.t1,self.get_F('vo'))
    
        I_ji_p = 2*np.einsum('jeim,me->ji',self.get_MO('ovoo'),self.t1)
        I_ji_p -= np.einsum('ejim,me->ji',self.get_MO('vooo'),self.t1)
        I_ji_p += np.einsum('efmi,mjef->ji',self.get_MO('vvoo'),asym_t2)
        #I_ji_p += 2*np.einsum('efmi,mjef->ji',self.get_MO('vvoo'),self.t2)
        #I_ji_p -= np.einsum('efmi,jmef->ji',self.get_MO('vvoo'),self.t2)
        I_ji_p += self.get_F('oo') - np.diag(np.diag(self.get_F('oo')))
        
        I_ji = np.einsum('ei,je->ji',I_ai,self.t1)
        I_ji += I_ji_p
            
        I_klij = np.einsum('efij,klef',self.get_MO('vvoo'),c_ijab)
        I_klij += np.einsum('ke,elij->klij',self.t1,self.get_MO('vooo'))
        I_klij += np.einsum('le,ekji->klij',self.t1,self.get_MO('vooo'))
        #I_klij += tmp
        #I_klij += tmp.swapaxes(0,1).swapaxes(2,3)
        I_klij += self.get_MO('oooo')
      
        I_jbia = -0.5*np.einsum('bime,mjae->jbia',self.get_MO('voov'),c_ijab)
        I_jbia -= np.einsum('jbim,ma->jbia',self.get_MO('ovoo'),self.t1)
        I_jbia += np.einsum('ebia,je->jbia',self.get_MO('vvov'),self.t1)
        I_jbia += self.get_MO('ovov')
        
        I_bjia = np.einsum('beim,mjea->bjia',self.get_MO('vvoo'),self.t2)
        I_bjia -= 0.5*np.einsum('ebim,mjea->bjia',self.get_MO('vvoo'),self.t2)
        print('bjia 1: ',np.linalg.norm(I_bjia))
        I_bjia -= 0.5*np.einsum('beim,mjae->bjia',self.get_MO('vvoo'),c_ijab)
        print('bjia 2: ',np.linalg.norm(I_bjia))
        #I_bjia = np.einsum('ebim,jmea->bjia',self.get_MO('vvoo'),self.t2)
        #I_bjia -= 0.5*np.einsum('bmie,jmae->bjia',self.get_MO('voov'),self.t2)
        #I_bjia -= 0.5*np.einsum('mbie,jmae->bjia',self.get_MO('ovov'),self.t2)
        
        I_bjia += np.einsum('beia,je->bjia',self.get_MO('vvov'),self.t1)
        I_bjia -= np.einsum('bjim,ma->bjia',self.get_MO('vooo'),self.t1)
        I_bjia += self.get_MO('voov')
        #I_bjia += (self.get_MO('voov').swapaxes(0,3).swapaxes(0,1)).reshape(8,5,5,8)
        
        I_ciab_p = -np.einsum('ciam,mb->ciab',self.get_MO('vovo'),self.t1)
        I_ciab_p -= np.einsum('ma,cimb->ciab',self.t1,self.get_MO('voov'))
        I_ciab_p += self.get_MO('vovv')
        
        x_bjia = np.einsum('beia,je->bjia',self.get_MO('vvov'),self.t1)
        
        I_jkia_p = np.einsum('efia,jkef->jkia',self.get_MO('vvov'),self.t2)
        I_jkia_p += np.einsum('je,ekia->jkia',self.t1,x_bjia)
        I_jkia_p += self.get_MO('ooov')
        
        return asym_t2, c_ijab, I_ai, I_ba, I_ji_p, I_ji, I_klij, I_jbia, I_bjia, I_ciab_p, I_jkia_p
        
    def update_amplitudes_restricted(self,ccd=False):
        asym_t2, c_ijab, I_ai, I_ba, I_ji_p, I_ji, I_klij, I_jbia, I_bjia, I_ciab_p, I_jkia_p = self.build_restricted_intermediates()
        #print(f'asym_t2:{asym_t2[0,0,0,0]}')
        #print(f'c_ijab:{c_ijab[0,0,0,0]}')
        #print(f'I_ai:{I_ai[0,0]}')
        #print(f'I_ba:{I_ba[0,0]}')
        #print(f'I_ji_p:{I_ji_p[0,0]}')
        #print(f'I_ji:{I_ji[0,0]}')
        #print(f'I_klij:{I_klij[0,0,0,0]}')
        #print(f'I_jbia:{I_jbia[0,0,0,0]}')
        #print(f'I_bjia:{I_bjia[0,0,0,0]}')
        #print(f'I_ciab_p:{I_ciab_p[0,0,0,0]}')
        #print(f'I_jkia_p:{I_jkia_p[0,0,0,0]}')
        #print(self.MO)
        
        #print('t1_old',self.t1)
        
        if not ccd:
            tmp_t1 = self.get_F('ov').copy()
            print(np.linalg.norm(tmp_t1))

            tmp_t1 += np.einsum('ea,ie->ia',I_ba,self.t1)
            print(np.linalg.norm(tmp_t1))

            tmp_t1 -= np.einsum('im,ma->ia',I_ji_p,self.t1)
            print(np.linalg.norm(tmp_t1))

            tmp_t1 += np.einsum('em,miea->ia',I_ai,asym_t2)
            print(np.linalg.norm(tmp_t1))

            tmp_t1 += 2.0*np.einsum('eima,me->ia',self.get_MO('voov'),self.t1)
            tmp_t1 -= np.einsum('eiam,me->ia',self.get_MO('vovo'),self.t1)
            print(np.linalg.norm(tmp_t1))

            #print('vvov:')
            #print(self.get_MO('vvov'))
            #print('asym_t2:')
            #print(asym_t2)
            tmp_t1 += np.einsum('efma,mief->ia',self.get_MO('vvov'),asym_t2)
            #print(tmp_t1)
            print(np.linalg.norm(tmp_t1))

            #print('vooo')
            #print(self.get_MO('vooo'))
            #print('asym_t2')
            #print(asym_t2)
            tmp_t1 -= np.einsum('eimn,mnea->ia',self.get_MO('vooo'),asym_t2)
            #print(tmp_t1)
            print(np.linalg.norm(tmp_t1))

            print(np.linalg.norm(tmp_t1/self.Dia))

            print('t1 done!')
        
        tmp_t2 = -np.einsum('imab,jm->ijab',self.t2,I_ji)
        t2_old = tmp_t2.copy()
        #print('Term 2')
        #print(np.linalg.norm(tmp_t2))
        #print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        tmp_t2 += np.einsum('ijae,eb->ijab',self.t2,I_ba)
        #print('Term 1')
        #print(np.linalg.norm(tmp_t2))
        #print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        
        tmp_t2 += 0.5*np.einsum('mnab,ijmn->ijab',c_ijab,I_klij)
        #print('Term 3')
        #print(np.linalg.norm(tmp_t2))
        #print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        tmp_t2 += 0.5*np.einsum('efab,ijef->ijab',self.get_MO('vvvv'),c_ijab)
        #print('Term 4')
        #print(np.linalg.norm(tmp_t2))
        #print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        tmp_t2 -= np.einsum('mjae,iemb->ijab',self.t2,I_jbia)
        #print('Term 5')
        #print(np.linalg.norm(tmp_t2))
        #print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
            
        tmp_t2 -= np.einsum('iema,mjeb->ijab',I_jbia,self.t2)
        #print('Term 6')
        #print(np.linalg.norm(tmp_t2))
        #print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        print(I_bjia.shape)
        #print(I_bjia)
        #print(I_bjia.swapaxes(1,3).swapaxes(1,2))
        #print(asym_t2)
        tmp_t2 += np.einsum('imae,ejmb->ijab',asym_t2,I_bjia)
        print('Term 7')
        print(np.linalg.norm(tmp_t2))
        print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        tmp_t2 += np.einsum('ie,ejab->ijab',self.t1,I_ciab_p)
        print('Term 8')
        print(np.linalg.norm(tmp_t2))
        print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        tmp_t2 -= np.einsum('ma,ijmb->ijab',self.t1,I_jkia_p)
        print('Term 9')
        print(np.linalg.norm(tmp_t2))
        print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        #tmp_t2 += np.einsum('jibe,ea->ijab',self.t2,I_ba)
        #tmp_t2 -= np.einsum('jmba,im->ijab',self.t2,I_ji)
        #tmp_t2 += 0.5*np.einsum('efba,jief->ijab',self.get_MO('vvvv'),c_ijab)
        #tmp_t2 += 0.5*np.einsum('mnba,jimn->ijab',c_ijab,I_klij)
        #tmp_t2 -= np.einsum('mibe,jema->ijab',self.t2,I_jbia)
        #tmp_t2 -= np.einsum('jemb,miea->ijab',I_jbia,self.t2)
        #tmp_t2 += np.einsum('mjeb,eima->ijab',asym_t2,I_bjia)
        #tmp_t2 += np.einsum('je,eiba->ijab',self.t1,I_ciab_p)
        #tmp_t2 -= np.einsum('mb,jima->ijab',self.t1,I_jkia_p)
        
        tmp_t2 += tmp_t2.swapaxes(0,1).swapaxes(2,3)
        print('Transposition')
        print(np.linalg.norm(tmp_t2))
        print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        tmp_t2 += self.get_MO('oovv')
        print('Term 0')
        print(np.linalg.norm(tmp_t2))
        print('delta: ',np.linalg.norm(tmp_t2-t2_old))
        t2_old = tmp_t2.copy()
        
        print('MBPT denom')
        print(np.linalg.norm(tmp_t2/self.Dijab))
        print('delta: ',np.linalg.norm(tmp_t2/self.Dijab-t2_old))
        t2_old = tmp_t2.copy()
        
        if not ccd:
            self.t1 = tmp_t1/self.Dia
        self.t2 = tmp_t2/self.Dijab
        
    def compute_corr_energy(self):
        c_ijab = self.t2.copy()
        c_ijab += np.einsum('ia,jb->ijab',self.t1,self.t1)
        CCSDcorr_E = 2.0 * np.einsum('abij,ijab->',self.get_MO('vvoo'),c_ijab)
        CCSDcorr_E -= np.einsum('baij,ijab->',self.get_MO('vvoo'),c_ijab)
        CCSDcorr_E += 2.0*np.einsum('ai,ia->',self.get_F('vo'),self.t1)

        self.ccsd_corr_e = CCSDcorr_E
        self.ccsd_e = self.rhf_e + self.ccsd_corr_e
        return CCSDcorr_E

    def compute_energy(self,
                       e_conv=1e-6,
                       r_conv=1e-4,
                       maxiter=100,
                       max_diis=9,
                       start_diis=1):

        ### Start Iterations
        ccsd_tstart = time.time()

        # Compute MP2 energy
        CCSDcorr_E_old = self.compute_corr_energy()
        print(
            "CCSD Iteration %3d: CCSD correlation = %.15f   dE = % .5E   MP2" %
            (0, CCSDcorr_E_old, -CCSDcorr_E_old))

        # Set up DIIS before iterations begin
        #diis_object = helper_diis(self.t1, self.t2, max_diis)
        
        # Iterate!
        for CCSD_iter in range(1, maxiter+1):
            self.update_amplitudes_restricted(ccd=False)

            # Compute CCSD correlation energy
            CCSDcorr_E = self.compute_corr_energy()

            # Print CCSD iteration information
            print(
                'CCSD Iteration %3d: CCSD correlation = %.15f   dE = % .5E'
                % (CCSD_iter, CCSDcorr_E, (CCSDcorr_E - CCSDcorr_E_old)))

            # Check convergence
            if (abs(CCSDcorr_E - CCSDcorr_E_old) < e_conv):
                print('\nCCSD has converged in %.3f seconds!' %
                      (time.time() - ccsd_tstart))
                return CCSDcorr_E

            # Update old energy
            CCSDcorr_E_old = CCSDcorr_E

            #  Add the new error vector
            #diis_object.add_error_vector(self.t1, self.t2)

            #if CCSD_iter >= start_diis:
            #    self.t1, self.t2 = diis_object.extrapolate(self.t1, self.t2)


# End HelperCCEnergy class

In [672]:
np.set_printoptions(precision=7, linewidth=200, suppress=True,threshold=sys.maxsize)

water = True

# Psi4 setup
psi4.set_memory('1 GB')
psi4.core.set_output_file('output.dat', False)
if water:
    mol = psi4.geometry("""
    O
    H 1 1.6
    H 1 1.6 2 104.45
    symmetry c1
    """)
    psi4.set_options({'basis': 'cc-pvdz'})
else:
    mol = psi4.geometry("""
    C
    C 1 1.5
    symmetry c1
    """)
    psi4.set_options({'basis': 'cc-pvdz'})

# roots per irrep must be set to do the eom calculation with psi4


# Compute CCSD energy for required integrals and T-amplitudes
rhf_e, rhf_wfn = psi4.energy('SCF', return_wfn=True)
ccsd = HelperCCEnergy(mol, rhf_e, rhf_wfn)
#print('MO',ccsd.MO)
ccsd.compute_energy()

ccsd_cor_e = ccsd.ccsd_corr_e
ccsd_tot_e = ccsd.ccsd_e
print("\n")
print("{}{}".format('RHF energy:'.ljust(30, '.'),
                    "{:16.15f}".format(rhf_e).rjust(20, '.')))
print("{}{}".format('Final CCSD correlation energy:'.ljust(30, '.'),
                    "{:16.15f}".format(ccsd_cor_e).rjust(20, '.')))
print("{}{}".format('Total CCSD energy:'.ljust(30, '.'),
                    "{:16.15f}".format(ccsd_tot_e).rjust(20, ',')))


Initalizing CCSD object...

Starting AO ->  MO transformation...
Size of the ERI tensor is 0.00 GB, 24 basis functions.
Building initial guess...

..initialized CCSD in 0.071 seconds.

CCSD Iteration   0: CCSD correlation = -0.262088125066937   dE =  2.62088E-01   MP2
bjia 1:  0.09753157676499649
bjia 2:  0.06501805550661494
0.0
0.0
0.0
0.0
0.0
0.13074400126629723
0.07151656243719465
0.044088039145886826
t1 done!
(19, 5, 5, 19)
Term 7
0.09295734891430431
delta:  0.1672869771904065
Term 8
0.09295734891430431
delta:  0.0
Term 9
0.09295734891430431
delta:  0.0
Transposition
0.14606936915573854
delta:  0.09295734891430431
Term 0
0.8532915659969252
delta:  0.8273185021709344
MBPT denom
0.3304239012307777
delta:  1.1299544260925887
CCSD Iteration   1: CCSD correlation = -0.252146462492656   dE =  9.94166E-03
bjia 1:  0.0919393409134158
bjia 2:  0.05320897581878659
0.0
0.001035263488491669
0.003969900344759047
0.0037896863335829334
0.012353029806482648
0.1191914903390173
0.07703069649022992


bjia 1:  0.11090701570611969
bjia 2:  0.05639243578900162
0.0
0.0029934050433970748
0.00857035681405232
0.007696966215409435
0.0171951333163435
0.13394808171490613
0.09621277792081269
0.07082492108081372
t1 done!
(19, 5, 5, 19)
Term 7
0.1358792799191507
delta:  0.1663268227340773
Term 8
0.13948575467304347
delta:  0.021239165845402412
Term 9
0.13917049517743277
delta:  0.023770299528421912
Transposition
0.24953725712004318
delta:  0.13917049517743277
Term 0
0.9294439712472252
delta:  0.8273185021709344
MBPT denom
0.4241217888960857
delta:  1.2893051812389724
CCSD Iteration  19: CCSD correlation = -0.278968327068082   dE = -3.14375E-06
bjia 1:  0.11091209576269709
bjia 2:  0.05639198621969158
0.0
0.0029944843286773422
0.00857236624394669
0.007698884545198117
0.017196806273173652
0.1339504175933098
0.09621633788137067
0.07083176040805898
t1 done!
(19, 5, 5, 19)
Term 7
0.13588629930288929
delta:  0.16632915736879703
Term 8
0.13949353004001463
delta:  0.021242767720316136
Term 9
0.13917868

In [239]:
ccsd.MO[0,1,0,1]

0.5596841555418639

In [291]:
sto3gh2mo = ccsd.MO.copy()

In [293]:
sto3gh2mo[1,0,0,1]

0.22953593610539547

In [294]:
sto3gh2mo[0,0,1,1]

0.22953593610539536

In [460]:
2*ccsd.get_MO('voov')-ccsd.get_MO('voov').swapaxes(0,3)

array([[[[ 0.0159018,  0.       , -0.0026538, ..., -0.027017 , -0.       ,  0.0200712],
         [-0.0128316,  0.       ,  0.0052799, ...,  0.042536 ,  0.       ,  0.0019168],
         [ 0.       , -0.       ,  0.       , ..., -0.       , -0.       ,  0.       ],
         [ 0.       , -0.0052521, -0.       , ...,  0.       , -0.0077181, -0.       ],
         [-0.0010625, -0.       , -0.0016109, ..., -0.0059132,  0.       , -0.0224597]],

        [[-0.0128316,  0.       ,  0.0011341, ..., -0.0002792, -0.       ,  0.004256 ],
         [ 0.0688175,  0.       , -0.0057851, ..., -0.0223412, -0.       ,  0.0336215],
         [ 0.       , -0.       ,  0.       , ...,  0.       ,  0.       ,  0.       ],
         [-0.       , -0.018076 ,  0.       , ...,  0.       ,  0.0024317, -0.       ],
         [-0.0322693, -0.       , -0.0012531, ..., -0.0160739,  0.       , -0.1317261]],

        [[ 0.       ,  0.       , -0.       , ..., -0.       ,  0.       , -0.       ],
         [ 0.       ,  0.   

In [472]:
ccsd.MO[5,0,0,5]

0.015901787761133428

In [473]:
np.argwhere(abs(ccsd.MO-0.015901787761133428)<1e-6)

array([[0, 0, 5, 5],
       [0, 5, 5, 0],
       [5, 0, 0, 5],
       [5, 5, 0, 0]])

array([-20.5326737,  -1.1954563,  -0.4935228,  -0.4728512,  -0.4496662,   0.0617293,   0.1137   ,   1.0942339,   1.1729444,   1.7596074,   1.8490881,   1.9071541,   2.6942562])

In [649]:
(ccsd.get_MO('voov').swapaxes(0,3).swapaxes(0,1)).reshape(8,5,5,8)

array([[[[ 0.0159018,  0.       , -0.0026538,  0.       ,  0.       , -0.027017 , -0.       ,  0.0200712],
         [-0.0128316,  0.       ,  0.002516 , -0.       , -0.       ,  0.0139925,  0.       ,  0.0034763],
         [ 0.       ,  0.       , -0.       ,  0.       ,  0.0037265, -0.       ,  0.       , -0.       ],
         [ 0.       ,  0.0020606,  0.       ,  0.0005755, -0.       , -0.       ,  0.0024663, -0.       ],
         [-0.0010625,  0.       ,  0.0016516, -0.       ,  0.       ,  0.0067773,  0.       ,  0.0098812]],

        [[ 0.       ,  0.0148605,  0.       ,  0.0036864,  0.       , -0.       ,  0.031481 ,  0.       ],
         [ 0.       , -0.016457 , -0.       , -0.0021342, -0.       ,  0.       , -0.0150507,  0.       ],
         [-0.       ,  0.       , -0.       ,  0.       ,  0.       , -0.       ,  0.       , -0.       ],
         [-0.0015957,  0.       , -0.0010128,  0.       , -0.       , -0.0007921,  0.       , -0.0095845],
         [ 0.       , -0.0043054, -